In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
sms = pd.read_csv('spam (2).csv',encoding= 'latin-1')
sms.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [4]:
sms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [6]:
sms = sms.drop(columns = ['Unnamed: 2','Unnamed: 3','Unnamed: 4'])

In [10]:
sms.rename(columns = {'v1':'label','v2':'message'},inplace = True)

In [11]:
sms.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [12]:
sms.shape

(5572, 2)

In [15]:
print(sms.label.value_counts())

label
ham     4825
spam     747
Name: count, dtype: int64


In [18]:
print(round(len(sms[sms['label']=='spam'])/len(sms.label),2)*100,'%')
print(round(len(sms[sms['label']=='ham'])/len(sms.label),2)*100,'%')

13.0 %
87.0 %


In [19]:
sms['length']= sms.message.str.len()
sms.head()

,label,message,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [20]:
# label encoding 0 and 1
sms.label=sms['label'].replace({'ham':0,'spam':1})

/tmp/ipykernel_345305/3200206911.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  sms.label=sms['label'].replace({'ham':0,'spam':1})


In [23]:
sms.message = sms.message.str.lower()

In [24]:
sms.head()

,label,message,length
0,0,"go until jurong point, crazy.. available only ...",111
1,0,ok lar... joking wif u oni...,29
2,1,free entry in 2 a wkly comp to win fa cup fina...,155
3,0,u dun say so early hor... u c already then say...,49
4,0,"nah i don't think he goes to usf, he lives aro...",61


In [25]:
# Replace email addresses with 'email'
sms['message'] = sms['message'].str.replace(r'^.+@[^\.].*\.[a-z]{2,}$',
                                 'emailaddress')

# Replace URLs with 'webaddress'
sms['message'] = sms['message'].str.replace(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$',
                                  'webaddress')

# Replace money symbols with 'moneysymb' (£ can by typed with ALT key + 156)
sms['message'] = sms['message'].str.replace(r'£|\$', 'dollers')
    
# Replace 10 digit phone numbers (formats include paranthesis, spaces, no spaces, dashes) with 'phonenumber'
sms['message'] = sms['message'].str.replace(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$',
                                  'phonenumber')

    
# Replace numbers with 'numbr'
sms['message'] = sms['message'].str.replace(r'\d+(\.\d+)?', 'numbr')

In [27]:
# Remove punctuation
sms['message'] = sms['message'].str.replace(r'[^\w\d\s]', ' ')

sms['message'] = sms['message'].str.replace(r',', ' ')

# Replace whitespace between terms with a single space
sms['message'] = sms['message'].str.replace(r'\s+', ' ')

# Remove leading and trailing whitespace
sms['message'] = sms['message'].str.replace(r'^\s+|\s+?$', '')

In [28]:
sms.head()

,label,message,length
0,0,go until jurong point crazy.. available only ...,111
1,0,ok lar... joking wif u oni...,29
2,1,free entry in 2 a wkly comp to win fa cup fina...,155
3,0,u dun say so early hor... u c already then say...,49
4,0,nah i don't think he goes to usf he lives aro...,61


In [30]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [31]:
# removing stop words
import string
import nltk
from nltk.corpus import stopwords


In [34]:
stop_words = set(stopwords.words('english')+['dont','doin','ure','2maro','etc','urs'])
sms.message = sms.message.apply(lambda x:"".join(term for term in x.split() if term not in stop_words and len(term)>2))
sms.head()

,label,message,length
0,0,jurongpointcrazy..availablebugisgreatworldbuff...,111
1,0,lar...jokingwifoni...,29
2,1,freeentrywklycompwincupfinaltkts21stmay2005.te...,155
3,0,dunsayearlyhor...alreadysay...,49
4,0,nahthinkgoesusflivesaroundthough,61


In [35]:
sms['new_len']=sms.message.str.len()

In [36]:
sms.head()

,label,message,length,new_len
0,0,jurongpointcrazy..availablebugisgreatworldbuff...,111,69
1,0,lar...jokingwifoni...,29,21
2,1,freeentrywklycompwincupfinaltkts21stmay2005.te...,155,114
3,0,dunsayearlyhor...alreadysay...,49,30
4,0,nahthinkgoesusflivesaroundthough,61,32


In [37]:
print(sms.length.sum())
print(sms.new_len.sum())

446422
265385


In [39]:
# tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score , confusion_matrix , classification_report

tf_vec = TfidfVectorizer()
naive = MultinomialNB()
features = tf_vec.fit_transform(sms.message)
X = features
y = sms.label

In [40]:
# train and predict
X_train,x_test,Y_train,y_test = train_test_split(X,y,test_size=0.2,random_state = 22)
naive.fit(X_train,Y_train)
y_pred = naive.predict(x_test)
print(accuracy_score(y_test,y_pred))

0.9103139013452914


In [41]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.91      1.00      0.95       971
           1       1.00      0.31      0.47       144

    accuracy                           0.91      1115
   macro avg       0.95      0.65      0.71      1115
weighted avg       0.92      0.91      0.89      1115



In [42]:
# confusion matrix
conf_mat = confusion_matrix(y_test,y_pred)
print(conf_mat)

[[971   0]
 [100  44]]
